In [1]:
println(sc.version)
val logger = org.apache.log4j.LogManager.getLogger("com.aravind.cust.satisfaction")

import org.apache.spark.sql._
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._

import org.apache.spark.sql._
import com.crealytics.spark.excel._
import com.crealytics.spark.excel.WorkbookReader
import com.microsoft.spark.notebook.msutils.MSFileInfo

In [3]:
//tlfs@xxx.dfs.core.windows.net
val defContainer = "tlfs"
val defStorageAccount = "xxx"

/*
account_name = "Your account name"
container_name = "Your container name"
relative_path = "Your path"

adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path)
*/

//abfss://tlfs@xxx.dfs.core.windows.net/poc/planre/BSO/part-00000-4e7f73e0-67dd-4773-a872-b1855e2629f2-c000.snappy.parquet
val storageAccount = "poc"
val container = "xxx"
val planReBsoPath = "planre/BSO/part-00000-4e7f73e0-67dd-4773-a872-b1855e2629f2-c000.snappy.parquet"
val planReBsoAbsolutePath = s"abfss://${storageAccount}@${container}.dfs.core.windows.net/${planReBsoPath}"

In [12]:
  import java.time._

  val files =   deepLs(new MSFileInfo(name = null, path = "abfss://poc@xxx.dfs.core.windows.net/planre/BSO", size = 0, isDir = true, isFile = false, System.currentTimeMillis()))
  println("files: "+files.length)
  val fileDf = spark.createDataFrame(spark.sparkContext.parallelize(files))
  //fileDf.show(truncate=false)
  display(fileDf)

In [3]:
logger.info(s"Reading from path: ${planReBsoAbsolutePath}")
val df = spark.read.parquet(planReBsoAbsolutePath)
df.show(5)

## 2022 P09 Daily Service Level 

In [4]:
val srvcLevlPath = "cust-srvc/2022 P09 Daily Service Level October 8-DB Worksheet-nonbinary.xlsx"
val srvcLevlAbsolutePath = s"abfss://${storageAccount}@${container}.dfs.core.windows.net/${srvcLevlPath}"

In [5]:


/*
treatEmptyValuesAsNulls = false,  // Optional, default: true
    setErrorCellsToFallbackValues = false, // Optional, default: false, where errors will be converted to null. If true, any ERROR cell values (e.g. #N/A) will be converted to the zero values of the column's data type.
    usePlainNumberFormat = false,  // Optional, default: false. If true, format the cells without rounding and scientific notations
    inferSchema = false,  // Optional, default: false
    addColorColumns = true,  // Optional, default: false
    timestampFormat = "MM-dd-yyyy HH:mm:ss",  // Optional, default: yyyy-mm-dd hh:mm:ss[.fffffffff]
    maxRowsInMemory = 20,  // Optional, default None. If set, uses a streaming reader which can help with big files (will fail if used with xls format files)
    maxByteArraySize = 2147483647,  // Optional, default None. See https://poi.apache.org/apidocs/5.0/org/apache/poi/util/IOUtils.html#setByteArrayMaxOverride-int-
    tempFileThreshold = 10000000, // Optional, default None. Number of bytes at which a zip entry is regarded as too large for holding in memory and the data is put in a temp file instead
    excerptSize = 10,  // Optional, default: 10. If set and if schema inferred, number of rows to infer schema from
    workbookPassword = "pass"  // Optional, default None. Requires unlimited strength JCE for older JVMs
*/
logger.info(s"Reading from path: ${srvcLevlAbsolutePath}")

val srvcLevlDf = spark.read.excel(
    header = true,  // Required
    dataAddress = "'DB'!A2", // Optional, default: "A1"
    inferSchema = true,
    maxRowsInMemory=100000
    
)//.schema(myCustomSchema) // Optional, default: Either inferred schema, or all columns are Strings
 .load(srvcLevlAbsolutePath)

In [6]:
println(s"Total rows: ${srvcLevlDf.count}")

## Consolidated Customer Database

In [16]:
val custDbPath = "cust-srvc/MT Consolidated Customer Database-nonbinary.xlsx"
val custDbAbsolutePath = s"abfss://${storageAccount}@${container}.dfs.core.windows.net/${custDbPath}"

logger.info(s"Reading from path: ${custDbAbsolutePath}")

val custDbDf = spark.read
    .excel(
        header = true,  // Required
        dataAddress = "'OUTLET'!A1", // Optional, default: "A1"
        inferSchema = true,
        maxRowsInMemory=200000)
    .load(custDbAbsolutePath)
    .cache()
    
println(s"Total rows: ${custDbDf.count}")

In [19]:
val schema: StructType = custDbDf.schema

addFileNameCol(custDbDf)(spark).select("OUTLET NO", "file_name").show(2)
//println("Infered schema for CustDbDf")
//custDbDf.printSchema()
//getType(schema)

//println(schema.prettyJson)

//val renamedCustDbDf = cleanColumnNames(custDbDf)

In [ ]:
val sheetNames = WorkbookReader( 
    Map("path" -> custDbAbsolutePath, "maxRowsInMemory" -> "100000"), 
    spark.sparkContext.hadoopConfiguration)
    .sheetNames